In [94]:
import h5py
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchsummary import summary

In [95]:
# 1. 加载数据
with h5py.File('C:/Users/liusi/Desktop/research/transformer/dataset.h5', 'r') as f:
    input_data = f['input_data'][:]  # 读取输入数据

    # 检查labels是否为标量数据集
    if f['labels'].shape == ():  # 如果是标量
        labels = f['labels'][()]  # 读取单个标量值
    else:
        labels = f['labels'][:]  # 读取数组

print("input_data shape:", input_data.shape)
print("labels shape:", labels.shape)



input_data shape: (20000, 1, 101, 2)
labels shape: (20000,)


In [96]:
# 2. 创建自定义数据集
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = torch.LongTensor(labels)  # 转换为长整型

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# 3. 划分训练和验证集
X_train, X_val, y_train, y_val = train_test_split(input_data, labels, test_size=0.2, random_state=42)

# 创建Dataset对象
train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_val, y_val)

# 4. 创建DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=64, shuffle=False)

In [116]:
class CNN(nn.Module):
    def __init__(self, num_classes, in_channels):
        super(CNN, self).__init__()
        self.cnn1 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=40, kernel_size=(2, 2), stride=1, padding=0), # 输出尺寸：(N, 40, H-1, W-1)
            nn.BatchNorm2d(40),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(1, 1), stride=(1, 1)) # 这个池化层可能不会改变尺寸，取决于实际需求
        )
        self.cnn2 = nn.Sequential(
            nn.Conv2d(in_channels=40, out_channels=1, kernel_size=(1, 1), stride=1, padding=0), # 输出尺寸：(N, 1, H-1, W-1)
            nn.BatchNorm2d(1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(1, 1), stride=(1, 1))  # 这个池化层可能不会改变尺寸，取决于实际需求
        )
        # 在此全连接层之前，需要知道上一层的输出尺寸
        # 假设上一层的输出尺寸是 (1, H', W')
        # 全连接层的输入尺寸将是 1 * H' * W'
        self.fc1 = nn.Linear(1 * 100 * 1, 2)  # 全连接层有2个神经元
        self.fc2 = nn.Linear(2, num_classes)  # 输出层

    def forward(self, x):
        out = self.cnn1(x)
        out = self.cnn2(out)
        out = out.view(out.size(0), -1)  # 展平
        out = F.relu(self.fc1(out))  # 应用ReLU激活函数
        out = self.fc2(out)
        return out


In [117]:
# 模型实例化
num_classes = 5  # 类别数
in_channel = 1   # 输入通道数

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN(num_classes=num_classes, in_channels=in_channel).to(device)
summary(model, input_size=(in_channel, 101, 2))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 40, 100, 1]             200
       BatchNorm2d-2           [-1, 40, 100, 1]              80
              ReLU-3           [-1, 40, 100, 1]               0
         MaxPool2d-4           [-1, 40, 100, 1]               0
            Conv2d-5            [-1, 1, 100, 1]              41
       BatchNorm2d-6            [-1, 1, 100, 1]               2
              ReLU-7            [-1, 1, 100, 1]               0
         MaxPool2d-8            [-1, 1, 100, 1]               0
            Linear-9                    [-1, 2]             202
           Linear-10                    [-1, 5]              15
Total params: 540
Trainable params: 540
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.13
Params size (MB): 0.00
Estimated Total Siz

##### 训练模型

In [125]:
learning_rate = 1e-4
criterior = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [134]:
num_epochs = 200
for epoch in range(num_epochs):
    for batch_idx, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # forward
        outputs = model(images)
        loss = criterior(outputs, labels)

        # backward
        optimizer.zero_grad() # 梯度清零
        loss.backward() # 反向传播

        # update
        optimizer.step() # 更新参数

        if (batch_idx+1) % 250 == 0:
            print(f"{epoch+1}/{num_epochs}, step {batch_idx+1}/{len(train_loader)}, loss = {loss.item():.4f}")

1/200, step 250/250, loss = 1.0340
2/200, step 250/250, loss = 0.8521
3/200, step 250/250, loss = 0.9797
4/200, step 250/250, loss = 0.9649
5/200, step 250/250, loss = 0.8700
6/200, step 250/250, loss = 0.9954
7/200, step 250/250, loss = 0.9822
8/200, step 250/250, loss = 0.9689
9/200, step 250/250, loss = 0.9217
10/200, step 250/250, loss = 1.0576
11/200, step 250/250, loss = 0.9150
12/200, step 250/250, loss = 0.9883
13/200, step 250/250, loss = 0.9912
14/200, step 250/250, loss = 0.8992
15/200, step 250/250, loss = 0.9374
16/200, step 250/250, loss = 1.0246
17/200, step 250/250, loss = 1.0108
18/200, step 250/250, loss = 1.1015
19/200, step 250/250, loss = 0.9877
20/200, step 250/250, loss = 1.1011
21/200, step 250/250, loss = 0.8611
22/200, step 250/250, loss = 0.9076
23/200, step 250/250, loss = 0.9029
24/200, step 250/250, loss = 0.9554
25/200, step 250/250, loss = 0.9402
26/200, step 250/250, loss = 0.9564
27/200, step 250/250, loss = 0.9494
28/200, step 250/250, loss = 0.9585
2

##### Model evaluation

In [132]:
# 1. 加载测试数据
with h5py.File('C:/Users/liusi/Desktop/research/transformer/dataset_test.h5', 'r') as f:
    test_input_data = f['input_data'][:]  # 假设测试集的特征数据键是 'input_data'
    test_labels = f['labels'][:]  # 假设测试集的标签数据键是 'labels'

# 2. 创建测试集 Dataset
test_dataset = CustomDataset(test_input_data, test_labels)

# 3. 创建测试集 DataLoader
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)  # 通常测试集不需要打乱

In [133]:
total = 0
correct = 0
for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)

    outputs = model(images)
    predicted = torch.argmax(outputs, dim=1)

    total += labels.shape[0]
    correct += (predicted == labels).sum().item()
    
print(f'{correct}/{total}, accuracy = {correct/total}')

10034/20000, accuracy = 0.5017
